In [ ]:
"""
1.对txt格式的年报，进行词频分析
2.分析数据转换为pandas.dataframe格式，合并到最终表格中
"""

In [1]:
#!jupyter nbconvert --to script word_freq.ipynb
import os
#探索活动	探索/探究/研究/研发/科研，搜寻/搜索，变化/变异，风险/冒险，实验，创新/设计，灵活，发现
#开发活动	利用/应用/开发，提炼/精炼/精益，稳定，选择，生产/投产，效率/成本，改进/改良，执行/实施
freq_words = ["探索","探究","研究","研发","科研","搜寻","搜索","变化","变异","风险","冒险","实验","创新","设计","灵活","发现",
              "利用","应用","开发","提炼","精炼","精益","稳定","选择","生产","投产","效率","成本","改进","改良","执行","实施"]
              
def find_files(base_dir,postfix='.pdf'):
    files = []
    for f in os.listdir(base_dir):
        sub_dir = os.path.join(base_dir,f)
        if not os.path.isdir(sub_dir):
            print('{sub_dir} is not dir')
            continue
        files += [os.path.join(sub_dir,f) for f in os.listdir(sub_dir) if f.endswith(postfix)]
    return files

base_dir='../txt/'
txt_files = find_files(base_dir,'.txt')
print(len(txt_files))
print(txt_files[:4])

21129
['../txt/2008/600298_2008.txt', '../txt/2008/600626_2008.txt', '../txt/2008/600330_2008.txt', '../txt/2008/600366_2008.txt']


In [2]:
# read file to one line
def read_content(file):
    try:   
        lines=[];
        with open(file, 'r', encoding='utf-8') as f:
            for line in f.readlines():                          #依次读取每行  
                line = line.strip()
                if len(line)>0:
                    lines.append(line)

        content="".join(lines)
        return content
    except:
        pass
    
    return ""


# test the function
file = '../scraping/content/300354_2015.txt'
content=read_content(file)
print(content[:256])


In [3]:
def word_freq(content,freq_words):
    freq_words_dict = {k:0 for k in freq_words}
    start = 0
    while start < len(content)-3:
        word = content[start:start+2] # 取两个词
        if word in freq_words_dict: freq_words_dict[word]+=1
        start+=2
    return freq_words_dict

# test the function
freq_words_dict= word_freq(content,freq_words)
print(freq_words_dict)

{'探索': 0, '探究': 0, '研究': 0, '研发': 0, '科研': 0, '搜寻': 0, '搜索': 0, '变化': 0, '变异': 0, '风险': 0, '冒险': 0, '实验': 0, '创新': 0, '设计': 0, '灵活': 0, '发现': 0, '利用': 0, '应用': 0, '开发': 0, '提炼': 0, '精炼': 0, '精益': 0, '稳定': 0, '选择': 0, '生产': 0, '投产': 0, '效率': 0, '成本': 0, '改进': 0, '改良': 0, '执行': 0, '实施': 0}


In [6]:
from tqdm import tqdm
import pandas as pd
records=[]
# for file in tqdm(txt_files[:4]):
for file in tqdm(txt_files):
    content=read_content(file)
    freq_words_dict= word_freq(content,freq_words)
    code,year = os.path.basename(file).replace('.txt','').split('_')
    freq_words_dict['Symbol']=code
    freq_words_dict['EndDate']=year+'-12-31'
    records.append(freq_words_dict)
    

df = pd.DataFrame(records)
df.head()

 12%|█▏        | 2507/21129 [02:15<17:09, 18.09it/s]

KeyboardInterrupt: 

In [28]:
# cols = df.columns.tolist()
# cols.remove('Symbol')
# cols.insert(0,'Symbol')
# df = df[cols]
# df.head()

,Symbol,EndDate,冒险,创新,利用,发现,变化,变异,实施,实验,...,生产,研发,研究,科研,稳定,精炼,精益,设计,选择,风险
0,300354,2015-12-31,0,13,4,5,7,0,27,5,...,41,31,19,2,4,0,0,6,4,21
1,603041,2017-12-31,0,8,4,6,13,0,23,0,...,60,22,12,1,38,0,1,7,1,42
2,002426,2013-12-31,0,2,1,1,2,0,0,0,...,2,3,0,0,0,0,0,0,0,2
3,300381,2016-12-31,0,9,6,8,6,0,22,4,...,115,32,16,6,18,0,0,1,5,36
4,002285,2012-12-31,0,0,3,1,5,0,2,0,...,0,0,1,0,0,0,0,2,1,5


In [5]:
filename='./研究数据_out.xlsx'
df2 = pd.read_excel(filename,index_col=None)

In [16]:
# 过滤中文字段，避免历史数据造成的干扰
df2.columns.tolist()
import re
chs_re = re.compile("[\u4e00-\u9fa5]+")
print(df2.columns.tolist())
filer_columns = list(filter(lambda x:not chs_re.match(x), df2.columns.tolist()))
print(filer_columns)
df2_ = df2[filer_columns[1:]]
df2_.head()

['Unnamed: 0', 'Symbol', 'EndDate', 'ShortName', 'IndustryName', 'IndustryCode', 'RegisterAddress', 'Zipcode', 'EstablishDate', 'Listexg', 'Ucpd', 'Ucsvr', 'Nshld', 'Empnum', 'RDPersonRatio', 'RDSpendSumRatio', 'Nshrttl', 'Nshrstt', 'Rat_SO', 'Nshrsms', 'TotalAssets', 'TotalLiability', 'IntangibleAsset', 'NetProfit', 'OperationProfit', 'F050201B', 'F050202B', 'F050203B', 'F050204C', 'F100501A', 'F100801A', 'F100802A', 'F100901A', 'F100902A', 'F100903A', 'F100904A', '备注', '冒险', '创新', '利用', '发现', '变化', '变异', '实施', '实验', '应用', '开发', '成本', '执行', '投产', '探究', '探索', '提炼', '搜寻', '搜索', '改良', '改进', '效率', '灵活', '生产', '研发', '研究', '科研', '稳定', '精炼', '精益', '设计', '选择', '风险']
['Unnamed: 0', 'Symbol', 'EndDate', 'ShortName', 'IndustryName', 'IndustryCode', 'RegisterAddress', 'Zipcode', 'EstablishDate', 'Listexg', 'Ucpd', 'Ucsvr', 'Nshld', 'Empnum', 'RDPersonRatio', 'RDSpendSumRatio', 'Nshrttl', 'Nshrstt', 'Rat_SO', 'Nshrsms', 'TotalAssets', 'TotalLiability', 'IntangibleAsset', 'NetProfit', 'OperationPro

,Symbol,EndDate,ShortName,IndustryName,IndustryCode,RegisterAddress,Zipcode,EstablishDate,Listexg,Ucpd,...,F050202B,F050203B,F050204C,F100501A,F100801A,F100802A,F100901A,F100902A,F100903A,F100904A
0,证券代码',统计截止日期',股票简称',行业名称',行业代码',注册具体地址',办公地址邮政编码',公司成立日期',上市交易所',实际控制人是否担任董事长或总经理',...,总资产净利润率（ROA）B',总资产净利润率（ROA）C',总资产净利润率（ROA）TTM',市值有形资产比',市值A',市值B',托宾Q值A',托宾Q值B',托宾Q值C',托宾Q值D'
1,000001,2017-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,0.007478,0.007478,0.007478,0.070564,3.25469e+12,3.25479e+12,0.0703,0.070566,1.00194,1.00574
2,000001,2016-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,0.008277,0.008277,0.008277,0.053125,2.9143e+12,2.90751e+12,0.052905,0.053127,0.984452,0.988582
3,000001,2015-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,0.009317,0.009317,0.009317,0.068774,2.51545e+12,2.51721e+12,0.068429,0.068772,1.00401,1.00906
4,000001,2014-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,0.009711,0.009711,0.009711,0.083259,2.22953e+12,2.23648e+12,0.082769,0.083258,1.02288,1.02893


In [18]:
df3 = df2_.merge(df,how='left',on=['Symbol','EndDate'])
df3.head()

,Symbol,EndDate,ShortName,IndustryName,IndustryCode,RegisterAddress,Zipcode,EstablishDate,Listexg,Ucpd,...,生产,研发,研究,科研,稳定,精炼,精益,设计,选择,风险
0,证券代码',统计截止日期',股票简称',行业名称',行业代码',注册具体地址',办公地址邮政编码',公司成立日期',上市交易所',实际控制人是否担任董事长或总经理',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000001,2017-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000001,2016-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000001,2015-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000001,2014-12-31,平安银行,货币金融服务,J66,广东省深圳市罗湖区深南东路5047号,518001,1987,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df3.to_excel(filename.replace('.xlsx','_words.xlsx'))